In [1]:
!pip install openpyxl lxml

In [2]:
import torch
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForTokenClassification,TrainingArguments, Trainer
import numpy as np
from tqdm.auto import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(1337)
torch.manual_seed(1337)

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_excel('CRA_train_1200.xlsx')

# Check the first few rows
print(df.head())

   Id                                             pr_txt Категория  \
0   1  Повышение кредитного рейтинга  Акционерного об...         A   
1   2  «Эксперт РА» подтвердил кредитный рейтинг комп...        BB   
2   3  НКР повысило кредитный рейтинг ООО "ОТЭКО-Порт...         A   
3   4  «Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...       AAA   
4   5  29 марта 2023 г. Ведущий рейтинговый аналитик ...       BBB   

  Уровень рейтинга  
0                A  
1               BB  
2                A  
3              AAA  
4              BBB  


In [4]:
mask = df['pr_txt'].str.len() > 200

# Use the boolean mask to select the rows that meet the condition
filtered_df = df[mask]

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
label_encoder = LabelEncoder()

# Encode the text labels
filtered_df['label'] = label_encoder.fit_transform(filtered_df['Уровень рейтинга'])

In [37]:
filtered_df['label'] = label_encoder.fit_transform(filtered_df['Категория'])

In [29]:
filtered_df['binary_label'] = 1-np.round(filtered_df['label']/6.0)

In [30]:
filtered_df['binary_label'].value_counts()

binary_label
1.0    782
0.0    418
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(filtered_df, test_size=0.15, random_state=42)

# Reset index for both dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [9]:
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification
from tokenizers import Encoding
#model_ckpt = "sberbank-ai/ruBert-base"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)

In [10]:
# Define a custom dataset class

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset


class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.data.loc[index, 'pr_txt'])
        label = int(self.data.loc[index, 'label'])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

# Create DataLoader for training and validation
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)

batch_size = 1

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)



In [11]:
from transformers import Trainer, TrainingArguments

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(label_encoder.classes_))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Define a function to compute F1 score
def compute_f1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return f1

# Modify the Trainer's compute_metrics function to include F1 score
def compute_metrics(p):
    preds = p.predictions
    labels = p.label_ids
    preds = preds.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds,average='weighted')
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [13]:

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir='./level_sbert_classification',
    evaluation_strategy='epoch',
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    save_steps=500,
    learning_rate=1e-5,
    weight_decay=0.01,
    push_to_hub=False,
    fp16 = True
)

# Create Trainer with modified compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Include F1 in metrics
)

# Start training
trainer.train()



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,2.575400,2.408653,0.169795,0.283333,0.205457
2,2.122100,2.014758,0.342636,0.394444,0.289136
3,1.561600,1.803406,0.426600,0.411111,0.386312
4,1.314300,1.613370,0.554550,0.516667,0.487406
5,0.951000,1.767775,0.586189,0.522222,0.512552
6,0.812400,1.878868,0.612337,0.538889,0.542200
7,0.614600,2.000560,0.606713,0.566667,0.563937
8,0.280800,2.182769,0.612049,0.566667,0.570686
9,0.165800,2.172813,0.627034,0.566667,0.575190
10,0.141200,2.215031,0.617898,0.566667,0.575791


/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

TrainOutput(global_step=10200, training_loss=1.0880403145621804, metrics={'train_runtime': 1236.7273, 'train_samples_per_second': 8.248, 'train_steps_per_second': 8.248, 'total_flos': 2684094206976000.0, 'train_loss': 1.0880403145621804, 'epoch': 10.0})

In [10]:
from transformers_interpret import SequenceClassificationExplainer
from transformers import BertForMaskedLM,BertTokenizerFast, pipeline, BatchEncoding,BertForTokenClassification,BertForSequenceClassification
from tokenizers import Encoding
#model_ckpt = "sberbank-ai/sbert_large_nlu_ru"
model_ckpt = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)

model = BertForSequenceClassification.from_pretrained('./category_bert_classification/checkpoint-1500/')

In [11]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

In [41]:
word_attributions = multiclass_explainer(text=str(val_df['pr_txt'][20])[:2000])

'LABEL_0'

In [42]:
html = multiclass_explainer.visualize()

In [34]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned BERT model
model = BertForSequenceClassification.from_pretrained('category_bert_classification/checkpoint-1000/') # Load from the directory where it was saved during training
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')



In [35]:
input_data = pd.read_excel('CRA_test_422.xlsx')  # Change the filename as needed


In [36]:
input_data

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«Эксперт РА» подтвердил рейтинг Трубной Металл...,NaN,NaN
1,1213,«Эксперт РА» подтвердил рейтинг компании «ГК «...,NaN,NaN
2,1214,«Эксперт РА» подтвердил рейтинг компании «Веле...,NaN,NaN
3,1215,АКРА подтвердило кредитный рейтинг Sanymon Cor...,NaN,NaN
4,1216,«Эксперт РА» подтвердил рейтинг компании «НьюТ...,NaN,NaN
...,...,...,...,...
417,1468,«Эксперт-Анализ» понизил прогноз кредитоспосо...,NaN,NaN
418,1472,Рейтинговое агентство «Эксперт» понизило рейт...,NaN,NaN
419,1475,«Эксперт РА» подтвердил рейтинг новой компани...,NaN,NaN
420,1486,"Финансовый анализ строительной компании ""Соз...",NaN,NaN


In [44]:
# Function to perform inference on a single text sample
def predict_text(text, model, tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=1024, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class = logits.argmax().item()
    return predicted_class

# Load the input data for inference (assuming it's in the same format)
input_data = pd.read_excel('CRA_test_422.xlsx')  # Change the filename as needed

# Perform inference on each text sample and add the predictions to a new column
input_data['predicted_label'] = input_data['pr_txt'].apply(lambda x: label_encoder.classes_[predict_text(x, model, tokenizer)])

# Display the results
print(input_data[['pr_txt', 'predicted_label']])

                                                pr_txt predicted_label
0    «Эксперт РА» подтвердил рейтинг Трубной Металл...               A
1    «Эксперт РА» подтвердил рейтинг компании «ГК «...               A
2    «Эксперт РА» подтвердил рейтинг компании «Веле...               A
3    АКРА подтвердило кредитный рейтинг Sanymon Cor...             BBB
4    «Эксперт РА» подтвердил рейтинг компании «НьюТ...             BBB
..                                                 ...             ...
417   «Эксперт-Анализ» понизил прогноз кредитоспосо...               A
418   Рейтинговое агентство «Эксперт» понизило рейт...              BB
419   «Эксперт РА» подтвердил рейтинг новой компани...               A
420   Финансовый анализ  строительной компании "Соз...               A
421   RAEX (Эксперт РА) присвоил рейтинг компании "...             BBB

[422 rows x 2 columns]


In [30]:
df_level

,Unnamed: 0,Id,pr_txt,Категория,Уровень рейтинга,predicted_label
0,0,1212,«Эксперт РА» подтвердил рейтинг Трубной Металл...,NaN,NaN,A
1,1,1213,«Эксперт РА» подтвердил рейтинг компании «ГК «...,NaN,NaN,A-
2,2,1214,«Эксперт РА» подтвердил рейтинг компании «Веле...,NaN,NaN,A-
3,3,1215,АКРА подтвердило кредитный рейтинг Sanymon Cor...,NaN,NaN,BBB-
4,4,1216,«Эксперт РА» подтвердил рейтинг компании «НьюТ...,NaN,NaN,BB
...,...,...,...,...,...,...
417,417,1468,«Эксперт-Анализ» понизил прогноз кредитоспосо...,NaN,NaN,A
418,418,1472,Рейтинговое агентство «Эксперт» понизило рейт...,NaN,NaN,BBB+
419,419,1475,«Эксперт РА» подтвердил рейтинг новой компани...,NaN,NaN,BBB-
420,420,1486,"Финансовый анализ строительной компании ""Соз...",NaN,NaN,AA


In [39]:
input_data.to_csv('category1.csv')

In [40]:
df_level = pd.read_csv('level.csv')
df_category = pd.read_csv('category1.csv')

In [41]:
sample_submission = pd.read_csv('submission.csv',sep =';')

In [42]:
sample_submission

,Id,Категория,Уровень рейтинга
0,1212,A,A-
1,1213,AA,AA
2,1214,A,A-
3,1215,A,A+
4,1216,BB,BB+
...,...,...,...
417,1468,AA,AA
418,1472,AAA,AAA
419,1475,A,A+
420,1486,AAA,AAA


In [46]:
sample_submission['Категория'] = df_category['predicted_label'].str.replace(r'[+-]', '', regex=True)
sample_submission['Уровень рейтинга'] = df_category['predicted_label']
sample_submission.to_csv('ans3.csv',sep = ';',index = False)

In [19]:
df_level['predicted_label']

0         A
1        A-
2        A-
3      BBB-
4        BB
       ... 
417       A
418    BBB+
419    BBB-
420      AA
421    BBB-
Name: predicted_label, Length: 422, dtype: object

In [20]:
df_category['predicted_label']

0        A
1      BBB
2       BB
3        A
4       BB
      ... 
417     AA
418    BBB
419    BBB
420     AA
421    BBB
Name: predicted_label, Length: 422, dtype: object